In [18]:
import random
import numpy as np
from pandas import DataFrame
from py2neo import Node,Relationship,Graph

In [2]:
# Fetching, Cleaning, and Storing User Names in a List
fh1 = open("Data/Names.txt", "r")
nameslist = []
for line in fh1:
    cleanline = line.strip()
    nameslist.append(cleanline)
    
#Getting the AgeList for each User
agelist = []
for i in range(0,250):
    agelist.append(random.randint(15,20))
for i in range(0,500):
    agelist.append(random.randint(21,30))
for i in range(0,200):
    agelist.append(random.randint(31,40))
for i in range(0,50):
    agelist.append(random.randint(41,50))
random.shuffle(agelist)

# Fetching the Gender of every User
genderlist = []
for i in range(0, 1000):
    if random.randint(1,2) == 1:
        genderlist.append('M')
    else:
        genderlist.append('F')
        
# Getting the Nationality of every User
country = ["US", "JAPAN", "FRANCE", "PORTUGAL", "CHILE", "TAIWAN", "SOUTH KOREA", "ITALY", "UAE",
               "COSTA RICA", "SLOVENIA", "BELGIUM", "SPAIN", "SINGAPORE", "UK", "GERMANY", "AUSTRIA", "CANADA",
               "IRELAND", "FINLAND", "NETHERLANDS", "NEW ZEALAND", "QATAR", "SWITZERLAND", "LUXEMBOURG",
               "ICELAND", "AUSTRALIA", "NORWAY", "SWEDEN", "DENMARK"]
countrylist = []
for i in range(0, 1000):
    countrylist.append(country[random.randint(0,29)])

In [3]:
frame1 = DataFrame({"Name": nameslist, 
                   "Age": agelist,
                    "Gender": genderlist,
                    "Country": countrylist})

In [4]:
frame1['Id'] = np.arange(1000)

In [5]:
frame2 = frame1.reindex(columns = ["Id", "Name", "Age", "Gender", "Country"])

In [6]:
count = 0
connectivitylist = []
for i in range(0, 1000):
    count = 0
    if 15 <= agelist[i] <= 20:
        while count < 15:
            id = random.randint(0, 999)
            if id == i:
                continue
            else:
                connectivitylist.append([i, id])
                count += 1

    elif 21 <= agelist[i] <= 30:
        while count < 25:
            id = random.randint(0, 999)
            if id == i:
                continue
            else:
                connectivitylist.append([i, id])
                count += 1

    elif 31 <= agelist[i] <= 40:
        while count < 10:
            id = random.randint(0, 999)
            if id == i:
                continue
            else:
                connectivitylist.append([i, id])
                count += 1

    else:
        while count < 5:
            id = random.randint(0, 999)
            if id == i:
                continue
            else:
                connectivitylist.append([i, id])
                count += 1

In [22]:
graph = Graph("http://localhost:7474/db/data/")
graph.delete_all()
graph_nodes = []

for i in range(1000):
    node = Node("Person", name=frame2.ix[i][1], age = str(frame2.ix[i][2]), 
                      gender= frame2.ix[i][3], country= frame2.ix[i][4])
    graph_nodes.append(node)
    graph.create(node)

In [24]:
relationship_list = []
for i in connectivitylist:
    relationship = Relationship(graph_nodes[i[0]], "KNOWS", graph_nodes[i[1]])
    relationship_list.append(relationship)
    graph.create(relationship)